In [ ]:
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import matplotlib.pyplot as plt
import imageio
import os

#ficheiros .py acessórios
import automatic_patching as patching_tool
import utils
import generate_data as generator_tool
import models as model_tool
import evaluator as evaluator

# Case-by-case

In [ ]:
patient_number = 38

In [ ]:
patient_folder_path = "patients/patient_{}/".format(patient_number)
image_paths = os.listdir(patient_folder_path)
not_images = []
for image in image_paths:
    if 't' in image:
        not_images.append(image)
for image in not_images:
    image_paths.remove(image)

In [ ]:
predictions_d= dict()
for image_view in image_paths:
    path = patient_folder_path+image_view
    saved_patches_paths = patching_tool.only_paths(path)
    df = pd.DataFrame({'paths':saved_patches_paths,'labels':'Test'})
    generated_data = generator_tool.generate(df)
    try:
        loaded_model = keras.models.load_model("D:/BCDR/vgg19_trained",compile=True)
    except OSError:
        print("Failed to load DL model!") 
    else:
        print("Successfully loaded DL model!")
        with tf.device('/gpu'):
            predictions = loaded_model.predict(generated_data)
            print("Successfully classified patches!")
            print('\n')

    predictions_image=[]
    for i in range(len(predictions)):
        predictions_image.append(predictions[i][0])

    predictions_d[image_view] = predictions_image

In [ ]:
if ground_truth[patient_number-1] == 0:
    gt = 'Normal'
else:
    gt = 'Suspicious'
print('Patient {} | {}'.format(patient_number,gt))
for image_view,i_classifications in predictions_d.items():
    ratios = []
    num_patches = len(i_classifications)
    positive_patches = 0
    for pred in i_classifications:
        if pred > 0.5:
            positive_patches += 1
    ratio = positive_patches/num_patches
    ratios.append(ratio)
    print(image_view[1:] +': Patches {} | Positive Patches {} | Ratio {:.2f}'.format(num_patches,positive_patches,ratio))
print('Average Ratio: ',round(average_ratio(ratios),2))

# Patching e Classificação

In [ ]:
patient_folder_path = "patients/"
image_paths = utils.sort_paths(os.listdir(patient_folder_path))

In [ ]:
try:
    loaded_model = keras.models.load_model("D:/BCDR/vgg19_trained",compile=True)
    print("Successfully loaded DL model!")
except OSError:
    print("Failed to load DL model!") 

In [ ]:
all_classifications = dict()
for patient in image_paths:    
    case_classification = dict()
    images = os.listdir(patient_folder_path+patient)
    for image_view in images:
        image_type = image_view.split(".")[0]
        path = patient_folder_path+patient+"/"+image_view
        saved_patches_paths = patching_tool.save_patches(path)
        df = pd.DataFrame({'paths':saved_patches_paths,'labels':'Test'})
        generated_data = generator_tool.generate(df)
        #predictions = model_tool.make_predict(generated_data)
        with tf.device('/gpu'):
            predictions = loaded_model.predict(generated_data)
            print('\n')
        predictions_image=[]
        for i in range(len(predictions)):
            predictions_image.append(predictions[i][0])
        case_classification[image_type] = predictions_image
    print("Successfully classified images from {}!".format(patient))
    print('\n')
    all_classifications[patient] = case_classification

In [ ]:
le = LabelEncoder()
ground_truth_csv = pd.read_csv('patients/gt_patients.csv')
ground_truth = le.fit_transform(list(ground_truth_csv['label']))

In [ ]:
report(all_classifications)

# Apenas Classificação

In [ ]:
patient_folder_path = "patients/"
image_paths = utils.sort_paths(os.listdir(patient_folder_path))

In [ ]:
try:
    loaded_model = keras.models.load_model("D:/BCDR/vgg19_trained",compile=True)
    print("Successfully loaded DL model!")
except OSError:
    print("Failed to load DL model!") 

In [ ]:
all_classifications = dict()
for patient in image_paths:    
    case_classification = dict()
    images = os.listdir(patient_folder_path+patient)
    not_images = []
    for image in images:
        if 't' in image:
            not_images.append(image)
    for image in not_images:
        images.remove(image)
    for image_view in images:
        image_type = image_view.split(".")[0]
        path = patient_folder_path+patient+"/"+image_view
        saved_patches_paths = patching_tool.only_paths(path)
        df = pd.DataFrame({'paths':saved_patches_paths,'labels':'Test'})
        generated_data = generator_tool.generate(df)
        with tf.device('/gpu'):
            predictions = loaded_model.predict(generated_data)
        predictions_image=[]
        for i in range(len(predictions)):
            predictions_image.append(predictions[i][0])
        case_classification[image_type] = predictions_image
    print("Successfully classified images from {}!".format(patient))
    print('\n')
    all_classifications[patient] = case_classification

In [ ]:
le = LabelEncoder()
ground_truth_csv = pd.read_csv('patients/gt_patients.csv')
ground_truth = le.fit_transform(list(ground_truth_csv['label']))

In [ ]:
evaluator.report(all_classifications,ground_truth)

## Single Threshold Classification

In [ ]:
predictions = evaluator.calculate_threshold_predictions(all_classifications,0.5)

In [ ]:
df,auc = evaluator.evaluate_predictions(image_paths,ground_truth,predictions)
print('AUC :',round(auc,4))

## Double Threshold Classification

In [ ]:
predictions,g_truth,patient_list = evaluator.calculate_case_predictions(all_classifications,list(ground_truth),image_paths,0.3,0.7)

In [ ]:
df,auc = evaluator.evaluate_predictions(patient_list,g_truth,predictions)
print('AUC :',round(auc,4))